In [2]:
import os
# set current directory as the repo base root
os.chdir('\\'.join(os.getcwd().split('\\')[:-1]))

In [3]:
from zeroCPR import engine

# initiate engine
myagent = engine.agent(groq_api_key='<your_api_key>')

c:\Users\ardit\miniconda3\envs\env1\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# encode products
import pandas as pd

# https://www.kaggle.com/datasets/ramzanzdemir/online-retail-gift-products
df = pd.read_excel('notebooks/df_raw.xlsx')
product_list = list(set(df['Description'].dropna().tolist()))
product_list = [x for x in product_list if isinstance(x, str)]
product_list = [x.strip() for x in product_list]
product_list

df_encoded = myagent.encode_products(product_list)
df_encoded.to_parquet('notebooks/df_encoded.parquet', index=None)

100%|██████████| 4678/4678 [02:01<00:00, 38.47it/s]


In [5]:
# in case you want to skip the encoding process, upload encoded products from parquet to save time
df_encoded = pd.read_parquet('notebooks/df_encoded.parquet')
myagent.upload_encoded_products(df_encoded)

### find complementaries of a single product

In [15]:
df_candidates, df_filtered = myagent.find_product_complementaries(['pizza'])
display(df_filtered)

,index,product_name,llm_product,recommended_product,reasoning,similarity_score,score
0,1608,pizza,Drink,wet,Not complementary,0.8,0
1,125,pizza,Breadsticks,CREAM SWEETHEART TRAYS,Complementary tableware,0.7,1
2,4086,pizza,Salad,FRUIT SALAD BAG CHARM,Not complementary,0.6,0
3,454,pizza,Wings,RED DRAGONFLY HELICOPTER,Not complementary,0.5,0
4,322,pizza,Dessert,RED PUDDING SPOON,Complementary utensil,0.4,1
5,1347,pizza,Toppings,PIZZA DISH SLICE,Complementary serving item,0.9,1
6,39,pizza,Dipping Sauce,BLUE BREAKFAST CUP AND SAUCER,Not complementary,0.8,0
7,472,pizza,Napkins,PACK OF 20 SPACEBOY NAPKINS,Complementary tableware,0.7,1
8,3900,pizza,Plates,SET/6 POSIES PAPER PLATES,Complementary tableware,0.6,1
9,2576,pizza,Cutlery,S/16 VINTAGE BLACK CUTLERY,Complementary utensil,0.5,1


In [16]:
#we can explore the comparison between complementary and similar products
print('complementary\t', df_filtered[df_filtered['score']==1]['recommended_product'].tolist())
print('similar\t\t', myagent.search_similar('pizza', k=len(df_filtered[df_filtered['score']==1])))

complementary	 ['CREAM SWEETHEART TRAYS', 'RED PUDDING SPOON', 'PIZZA DISH SLICE', 'PACK OF 20 SPACEBOY NAPKINS', 'SET/6 POSIES PAPER PLATES', 'S/16 VINTAGE BLACK CUTLERY', 'SET OF 2 ROUND TINS DUTCH CHEESE', 'JUMBO BAG STRAWBERRY', 'BREAD BIN, DINER STYLE, MINT']
similar		 ['PIZZA SLICE DISH', 'PIZZA DISH SLICE', 'PARTY PIZZA DISH BLUE POLKADOT', 'PARTY PIZZA DISH PINK POLKADOT', 'PARTY PIZZA DISH GREEN POLKADOT', 'PIZZA PLATE IN BOX', 'PARTY PIZZA DISH RED+WHITE SPOT', 'PARTY PIZZA DISH RED WHITE SPOT', 'PARTY PIZZA DISH PINK+WHITE SPOT']


### find the complementary of 10 products

In [18]:
df_complementaries = myagent.find_product_complementaries(product_list[50:60])
df_complementaries.to_parquet('notebooks/df_complementaries.parquet', index=None)

** SET 3 WICKER LOG BASKETS
** FUNKY GIRLZ ASST MAGNETIC MEMO PAD
** BLACK GEMSTONE BRACELET
ERR
** TOY TIDY PINK POLKADOT
** CROCHET WHITE RABBIT KEYRING
** MAGNETS PACK OF 4 HOME SWEET HOME
** POPCORN HOLDER , SMALL
** FLOWER BURST SILVER RING GREEN
** CRYSTAL DIAMANTE EXPANDABLE RING
** ASSORTED EASTER GIFT TAGS


In [19]:
df_complementaries

,index,product_name,llm_product,recommended_product,reasoning,similarity_score,score
0,4567,SET 3 WICKER LOG BASKETS,Fireplace Accessories,SMALL SINGLE FLAME CANDLE HOLDER,complementary: similar home decor item,0.8,1
1,4137,SET 3 WICKER LOG BASKETS,Firewood,CHRISTMAS RETROSPOT ANGEL WOOD,not complementary: seasonal item,0.7,0
2,259,SET 3 WICKER LOG BASKETS,Kindling,POTTERING MUG,not complementary: different category,0.6,0
3,687,SET 3 WICKER LOG BASKETS,Fire Starter,FLAMINGO LIGHTS,complementary: similar outdoor decoration item,0.5,1
4,50,SET 3 WICKER LOG BASKETS,Log Carrier,SET 3 WICKER LOG BASKETS,already purchased,0.4,0
...,...,...,...,...,...,...,...
0,59,ASSORTED EASTER GIFT TAGS,EASTER CARDS,CREAM FELT EASTER EGG BASKET,can be used with Easter gift tags to decorate ...,0.8,1
1,1918,ASSORTED EASTER GIFT TAGS,EASTER BASKET FILLERS,EMPIRE GIFT WRAP,can be used with Easter gift tags to wrap East...,0.6,1
2,2731,ASSORTED EASTER GIFT TAGS,GIFT WRAP,RIBBONS PURSE,can be used with Easter gift tags to add a dec...,0.5,1
3,69,ASSORTED EASTER GIFT TAGS,RIBBON OR BOW,ANIMAL STICKERS,not directly related to Easter gift tags,0.5,0
